In [299]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import datetime as dt
import calendar 
import numpy as np

In [300]:
#  Import and read the Metro_Interstate_Traffic_Volume.csv.

Traffic_analysis_df = pd.read_csv("Metro_Interstate_Traffic_Volume.csv")
Traffic_analysis_df.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
0,None,288.28,0.0,0.0,40,Clouds,scattered clouds,10/2/2012 9:00,5545
1,None,289.36,0.0,0.0,75,Clouds,broken clouds,10/2/2012 10:00,4516
2,None,289.58,0.0,0.0,90,Clouds,overcast clouds,10/2/2012 11:00,4767
3,None,290.13,0.0,0.0,90,Clouds,overcast clouds,10/2/2012 12:00,5026
4,None,291.14,0.0,0.0,75,Clouds,broken clouds,10/2/2012 13:00,4918


In [301]:
#  Import and read the 2014-2020_AllUserData_4Website.xlsx.

bike_df = pd.read_excel('2014-2020_AllUserData_4Website.xlsx')

bike_df.head()


,site,agency,lat,long,city,county,mndot_district,mode,date_day,doy,...,imputed,total_yearly_imputed_days,facility_type,on/off_road,install_year,trunk_hwy,us_bikeroute,device,direction,technology
0,Brown's Creek ST - Stillwater,MnDNR,45.064023,-92.805888,Stillwater,Washington,M,Bicyclist,2018-01-02,2,...,True,256,Shared Use Path,Off,2018,No,None,Eco-Multi,North/South,Infra-red & Inductive loops
1,Brown's Creek ST - Stillwater,MnDNR,45.064023,-92.805888,Stillwater,Washington,M,Bicyclist,2018-01-03,3,...,True,256,Shared Use Path,Off,2018,No,None,Eco-Multi,North/South,Infra-red & Inductive loops
2,Brown's Creek ST - Stillwater,MnDNR,45.064023,-92.805888,Stillwater,Washington,M,Bicyclist,2018-01-04,4,...,True,256,Shared Use Path,Off,2018,No,None,Eco-Multi,North/South,Infra-red & Inductive loops
3,Brown's Creek ST - Stillwater,MnDNR,45.064023,-92.805888,Stillwater,Washington,M,Bicyclist,2018-01-05,5,...,True,256,Shared Use Path,Off,2018,No,None,Eco-Multi,North/South,Infra-red & Inductive loops
4,Brown's Creek ST - Stillwater,MnDNR,45.064023,-92.805888,Stillwater,Washington,M,Bicyclist,2018-01-06,6,...,True,256,Shared Use Path,Off,2018,No,None,Eco-Multi,North/South,Infra-red & Inductive loops


In [302]:
# Drop the non-beneficial ID columns, 'weather_main' and 'weather_description'.

Traffic_analysis_df = Traffic_analysis_df.drop(['weather_main','weather_description'], axis=1)
Traffic_analysis_df

,holiday,temp,rain_1h,snow_1h,clouds_all,date_time,traffic_volume
0,None,288.28,0.0,0.0,40,10/2/2012 9:00,5545
1,None,289.36,0.0,0.0,75,10/2/2012 10:00,4516
2,None,289.58,0.0,0.0,90,10/2/2012 11:00,4767
3,None,290.13,0.0,0.0,90,10/2/2012 12:00,5026
4,None,291.14,0.0,0.0,75,10/2/2012 13:00,4918
...,...,...,...,...,...,...,...
48199,None,283.45,0.0,0.0,75,9/30/2018 19:00,3543
48200,None,282.76,0.0,0.0,90,9/30/2018 20:00,2781
48201,None,282.73,0.0,0.0,90,9/30/2018 21:00,2159
48202,None,282.09,0.0,0.0,90,9/30/2018 22:00,1450


In [303]:
Traffic_analysis_df.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,date_time,traffic_volume
0,None,288.28,0.0,0.0,40,10/2/2012 9:00,5545
1,None,289.36,0.0,0.0,75,10/2/2012 10:00,4516
2,None,289.58,0.0,0.0,90,10/2/2012 11:00,4767
3,None,290.13,0.0,0.0,90,10/2/2012 12:00,5026
4,None,291.14,0.0,0.0,75,10/2/2012 13:00,4918


In [304]:
Traffic_analysis_features = Traffic_analysis_df.copy()

In [305]:
# #Extracting features from date_time variable
Traffic_analysis_features['date_time'] = pd.to_datetime(Traffic_analysis_features.date_time)
Traffic_analysis_features['weekday'] = Traffic_analysis_features.date_time.dt.weekday
Traffic_analysis_features['date'] = Traffic_analysis_features.date_time.dt.date
Traffic_analysis_features['hour'] = Traffic_analysis_features.date_time.dt.hour
Traffic_analysis_features['month'] = Traffic_analysis_features.date_time.dt.month
Traffic_analysis_features['year'] = Traffic_analysis_features.date_time.dt.year
#Monday is 0 and Sunday is 6

In [306]:
Traffic_analysis_features = Traffic_analysis_features.drop(['date_time','hour','month','year'], axis=1)
Traffic_analysis_features

,holiday,temp,rain_1h,snow_1h,clouds_all,traffic_volume,weekday,date
0,None,288.28,0.0,0.0,40,5545,1,2012-10-02
1,None,289.36,0.0,0.0,75,4516,1,2012-10-02
2,None,289.58,0.0,0.0,90,4767,1,2012-10-02
3,None,290.13,0.0,0.0,90,5026,1,2012-10-02
4,None,291.14,0.0,0.0,75,4918,1,2012-10-02
...,...,...,...,...,...,...,...,...
48199,None,283.45,0.0,0.0,75,3543,6,2018-09-30
48200,None,282.76,0.0,0.0,90,2781,6,2018-09-30
48201,None,282.73,0.0,0.0,90,2159,6,2018-09-30
48202,None,282.09,0.0,0.0,90,1450,6,2018-09-30


In [307]:
Traffic_analysis_features['date'] = pd.to_datetime(Traffic_analysis_features['date'], format='%Y-%m-%d')
Traffic_analysis_features

,holiday,temp,rain_1h,snow_1h,clouds_all,traffic_volume,weekday,date
0,None,288.28,0.0,0.0,40,5545,1,2012-10-02
1,None,289.36,0.0,0.0,75,4516,1,2012-10-02
2,None,289.58,0.0,0.0,90,4767,1,2012-10-02
3,None,290.13,0.0,0.0,90,5026,1,2012-10-02
4,None,291.14,0.0,0.0,75,4918,1,2012-10-02
...,...,...,...,...,...,...,...,...
48199,None,283.45,0.0,0.0,75,3543,6,2018-09-30
48200,None,282.76,0.0,0.0,90,2781,6,2018-09-30
48201,None,282.73,0.0,0.0,90,2159,6,2018-09-30
48202,None,282.09,0.0,0.0,90,1450,6,2018-09-30


In [308]:
def modify_holiday(x):
    if x == 'None':
        return False
    else:
        return True
Traffic_analysis_features['holiday'] = Traffic_analysis_features['holiday'].map(modify_holiday)
Traffic_analysis_features

,holiday,temp,rain_1h,snow_1h,clouds_all,traffic_volume,weekday,date
0,False,288.28,0.0,0.0,40,5545,1,2012-10-02
1,False,289.36,0.0,0.0,75,4516,1,2012-10-02
2,False,289.58,0.0,0.0,90,4767,1,2012-10-02
3,False,290.13,0.0,0.0,90,5026,1,2012-10-02
4,False,291.14,0.0,0.0,75,4918,1,2012-10-02
...,...,...,...,...,...,...,...,...
48199,False,283.45,0.0,0.0,75,3543,6,2018-09-30
48200,False,282.76,0.0,0.0,90,2781,6,2018-09-30
48201,False,282.73,0.0,0.0,90,2159,6,2018-09-30
48202,False,282.09,0.0,0.0,90,1450,6,2018-09-30


In [309]:
def number_holiday(x):
    if x == 'False':
        return 1
    else:
        return 0
Traffic_analysis_features['holiday'] = Traffic_analysis_features['holiday'].map(number_holiday)
Traffic_analysis_features

,holiday,temp,rain_1h,snow_1h,clouds_all,traffic_volume,weekday,date
0,0,288.28,0.0,0.0,40,5545,1,2012-10-02
1,0,289.36,0.0,0.0,75,4516,1,2012-10-02
2,0,289.58,0.0,0.0,90,4767,1,2012-10-02
3,0,290.13,0.0,0.0,90,5026,1,2012-10-02
4,0,291.14,0.0,0.0,75,4918,1,2012-10-02
...,...,...,...,...,...,...,...,...
48199,0,283.45,0.0,0.0,75,3543,6,2018-09-30
48200,0,282.76,0.0,0.0,90,2781,6,2018-09-30
48201,0,282.73,0.0,0.0,90,2159,6,2018-09-30
48202,0,282.09,0.0,0.0,90,1450,6,2018-09-30


In [310]:
Traffic_analysis_features["holiday"] = Traffic_analysis_features["holiday"].astype(int)
Traffic_analysis_features

,holiday,temp,rain_1h,snow_1h,clouds_all,traffic_volume,weekday,date
0,0,288.28,0.0,0.0,40,5545,1,2012-10-02
1,0,289.36,0.0,0.0,75,4516,1,2012-10-02
2,0,289.58,0.0,0.0,90,4767,1,2012-10-02
3,0,290.13,0.0,0.0,90,5026,1,2012-10-02
4,0,291.14,0.0,0.0,75,4918,1,2012-10-02
...,...,...,...,...,...,...,...,...
48199,0,283.45,0.0,0.0,75,3543,6,2018-09-30
48200,0,282.76,0.0,0.0,90,2781,6,2018-09-30
48201,0,282.73,0.0,0.0,90,2159,6,2018-09-30
48202,0,282.09,0.0,0.0,90,1450,6,2018-09-30


In [311]:
Traffic_analysis_features.dtypes

holiday                    int32
temp                     float64
rain_1h                  float64
snow_1h                  float64
clouds_all                 int64
traffic_volume             int64
weekday                    int64
date              datetime64[ns]
dtype: object

In [312]:
# Converting Kelvin to Celsius

Traffic_analysis_features['temp'] = (Traffic_analysis_features['temp']-273.15)
Traffic_analysis_features.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,traffic_volume,weekday,date
0,0,15.13,0.0,0.0,40,5545,1,2012-10-02
1,0,16.21,0.0,0.0,75,4516,1,2012-10-02
2,0,16.43,0.0,0.0,90,4767,1,2012-10-02
3,0,16.98,0.0,0.0,90,5026,1,2012-10-02
4,0,17.99,0.0,0.0,75,4918,1,2012-10-02


In [313]:
# Converting Celsius to Fahrenheit

Traffic_analysis_features['temp'] = (Traffic_analysis_features['temp'] * 1.8 + 32)
Traffic_analysis_features

,holiday,temp,rain_1h,snow_1h,clouds_all,traffic_volume,weekday,date
0,0,59.234,0.0,0.0,40,5545,1,2012-10-02
1,0,61.178,0.0,0.0,75,4516,1,2012-10-02
2,0,61.574,0.0,0.0,90,4767,1,2012-10-02
3,0,62.564,0.0,0.0,90,5026,1,2012-10-02
4,0,64.382,0.0,0.0,75,4918,1,2012-10-02
...,...,...,...,...,...,...,...,...
48199,0,50.540,0.0,0.0,75,3543,6,2018-09-30
48200,0,49.298,0.0,0.0,90,2781,6,2018-09-30
48201,0,49.244,0.0,0.0,90,2159,6,2018-09-30
48202,0,48.092,0.0,0.0,90,1450,6,2018-09-30


In [314]:
start_date = '2015-12-31'
end_date = '2018-9-30'

mask = (Traffic_analysis_features['date'] > start_date) & (Traffic_analysis_features['date'] <= end_date)

Traffic_analysis_features = Traffic_analysis_features.loc[mask]
Traffic_analysis_features

,holiday,temp,rain_1h,snow_1h,clouds_all,traffic_volume,weekday,date
20344,0,19.022,0.0,0.0,90,1513,4,2016-01-01
20345,0,19.022,0.0,0.0,90,1513,4,2016-01-01
20346,0,19.130,0.0,0.0,90,1550,4,2016-01-01
20347,0,19.148,0.0,0.0,90,719,4,2016-01-01
20348,0,16.970,0.0,0.0,90,533,4,2016-01-01
...,...,...,...,...,...,...,...,...
48199,0,50.540,0.0,0.0,75,3543,6,2018-09-30
48200,0,49.298,0.0,0.0,90,2781,6,2018-09-30
48201,0,49.244,0.0,0.0,90,2159,6,2018-09-30
48202,0,48.092,0.0,0.0,90,1450,6,2018-09-30


In [315]:
# Drop the non-beneficial ID columns, 'site','agency','lat','long','city','mndot_district','mode','doy','imputed''.
bike_df = bike_df.drop(['site','agency','lat','long','city','mndot_district','doy','imputed',], axis=1)
bike_df.head

<bound method NDFrame.head of             county        mode   date_day      total  PRCP  TMAX  \
0       Washington   Bicyclist 2018-01-02  21.586591  0.00  15.0   
1       Washington   Bicyclist 2018-01-03  17.323078  0.00  11.0   
2       Washington   Bicyclist 2018-01-04  12.813717  0.00   2.0   
3       Washington   Bicyclist 2018-01-05  13.651215  0.00   3.0   
4       Washington   Bicyclist 2018-01-06  21.927954  0.00  14.0   
...            ...         ...        ...        ...   ...   ...   
78425  Saint Louis  Pedestrian 2020-12-27  31.000000  0.00  23.0   
78426  Saint Louis  Pedestrian 2020-12-28  19.000000  0.00  17.0   
78427  Saint Louis  Pedestrian 2020-12-29  19.000000  0.15  21.0   
78428  Saint Louis  Pedestrian 2020-12-30  30.000000  0.00  24.0   
78429  Saint Louis  Pedestrian 2020-12-31  34.000000  0.00  19.0   

       total_yearly_imputed_days    facility_type on/off_road  install_year  \
0                            256  Shared Use Path         Off          201

In [316]:
bike_df = bike_df.drop(['PRCP','TMAX','total_yearly_imputed_days','on/off_road','install_year','trunk_hwy',
                        'us_bikeroute','device','direction','technology'], axis=1)
bike_df

,county,mode,date_day,total,facility_type
0,Washington,Bicyclist,2018-01-02,21.586591,Shared Use Path
1,Washington,Bicyclist,2018-01-03,17.323078,Shared Use Path
2,Washington,Bicyclist,2018-01-04,12.813717,Shared Use Path
3,Washington,Bicyclist,2018-01-05,13.651215,Shared Use Path
4,Washington,Bicyclist,2018-01-06,21.927954,Shared Use Path
...,...,...,...,...,...
78425,Saint Louis,Pedestrian,2020-12-27,31.000000,Shared Use Path
78426,Saint Louis,Pedestrian,2020-12-28,19.000000,Shared Use Path
78427,Saint Louis,Pedestrian,2020-12-29,19.000000,Shared Use Path
78428,Saint Louis,Pedestrian,2020-12-30,30.000000,Shared Use Path


In [317]:
Ramsey_county_df = bike_df[bike_df["county"] == "Ramsey"]
Ramsey_county_df

,county,mode,date_day,total,facility_type
68204,Ramsey,Bicyclist,2016-01-01,16.472980,Sidewalk & Bike Facility
68205,Ramsey,Bicyclist,2016-01-02,14.725153,Sidewalk & Bike Facility
68206,Ramsey,Bicyclist,2016-01-03,13.798586,Sidewalk & Bike Facility
68207,Ramsey,Bicyclist,2016-01-04,16.097296,Sidewalk & Bike Facility
68208,Ramsey,Bicyclist,2016-01-05,18.489358,Sidewalk & Bike Facility
...,...,...,...,...,...
74045,Ramsey,Bicyclist,2020-12-27,218.000000,Bike Lane
74046,Ramsey,Bicyclist,2020-12-28,229.000000,Bike Lane
74047,Ramsey,Bicyclist,2020-12-29,203.000000,Bike Lane
74048,Ramsey,Bicyclist,2020-12-30,252.000000,Bike Lane


In [318]:
Bike_start_date = '2015-12-31'
Bike_end_date = '2018-9-30'

mask = (Ramsey_county_df['date_day'] > start_date) & (Ramsey_county_df['date_day'] <= end_date)

Ramsey_county_df = Ramsey_county_df.loc[mask]
Ramsey_county_df

,county,mode,date_day,total,facility_type
68204,Ramsey,Bicyclist,2016-01-01,16.472980,Sidewalk & Bike Facility
68205,Ramsey,Bicyclist,2016-01-02,14.725153,Sidewalk & Bike Facility
68206,Ramsey,Bicyclist,2016-01-03,13.798586,Sidewalk & Bike Facility
68207,Ramsey,Bicyclist,2016-01-04,16.097296,Sidewalk & Bike Facility
68208,Ramsey,Bicyclist,2016-01-05,18.489358,Sidewalk & Bike Facility
...,...,...,...,...,...
73222,Ramsey,Bicyclist,2018-09-26,940.000000,Bike Lane
73223,Ramsey,Bicyclist,2018-09-27,759.000000,Bike Lane
73224,Ramsey,Bicyclist,2018-09-28,772.000000,Bike Lane
73225,Ramsey,Bicyclist,2018-09-29,569.000000,Bike Lane


In [319]:
bike_grouped_dates = Ramsey_county_df.groupby('date_day').aggregate({'total':'sum',})
bike_grouped_dates

,total
date_day,
2016-01-01,539.633372
2016-01-02,489.464993
2016-01-03,414.111465
2016-01-04,611.207676
2016-01-05,667.725920
...,...
2018-09-26,1612.000000
2018-09-27,1342.000000
2018-09-28,1330.000000


In [320]:
grouped_dates = Traffic_analysis_features.groupby('date').aggregate({'traffic_volume':'sum','rain_1h':'sum','snow_1h':'sum','holiday':'sum'
                                                                    ,'clouds_all':'sum','temp':'mean'})
grouped_dates

,traffic_volume,rain_1h,snow_1h,holiday,clouds_all,temp
date,,,,,,
2016-01-01,34489,0.00,0.0,0,755,20.984947
2016-01-02,41018,0.00,0.0,0,185,20.935000
2016-01-03,28628,0.00,0.0,0,888,26.431077
2016-01-04,81891,0.00,0.0,0,692,13.949429
2016-01-05,61120,0.00,0.0,0,831,24.780941
...,...,...,...,...,...,...
2018-09-26,88627,0.25,0.0,0,956,51.522500
2018-09-27,94434,0.50,0.0,0,1530,55.613120
2018-09-28,92518,0.00,0.0,0,630,47.189000


In [321]:
combined_df = pd.concat([grouped_dates, bike_grouped_dates], axis=1)
combined_df

,traffic_volume,rain_1h,snow_1h,holiday,clouds_all,temp,total
2016-01-01,34489,0.00,0.0,0,755,20.984947,539.633372
2016-01-02,41018,0.00,0.0,0,185,20.935000,489.464993
2016-01-03,28628,0.00,0.0,0,888,26.431077,414.111465
2016-01-04,81891,0.00,0.0,0,692,13.949429,611.207676
2016-01-05,61120,0.00,0.0,0,831,24.780941,667.725920
...,...,...,...,...,...,...,...
2018-09-26,88627,0.25,0.0,0,956,51.522500,1612.000000
2018-09-27,94434,0.50,0.0,0,1530,55.613120,1342.000000
2018-09-28,92518,0.00,0.0,0,630,47.189000,1330.000000
2018-09-29,76242,0.25,0.0,0,1434,40.973360,903.000000


In [322]:
# Split our preprocessed data into our features and target arrays
y = combined_df["total"].values
X = combined_df.drop(["total"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=50)

In [323]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [324]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  52
hidden_nodes_layer2 = 26

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 52)                364       
_________________________________________________________________
dense_20 (Dense)             (None, 26)                1378      
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 27        
Total params: 1,769
Trainable params: 1,769
Non-trainable params: 0
_________________________________________________________________


In [325]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4020)

In [326]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=30,callbacks=[cp_callback])

Epoch 1/30
24/24 [==============================] - 0s 799us/step - loss: -10.9317 - accuracy: 0.0000e+00
Epoch 2/30
24/24 [==============================] - 0s 774us/step - loss: -668.5281 - accuracy: 0.0000e+00
Epoch 3/30
24/24 [==============================] - 0s 702us/step - loss: -1708.2565 - accuracy: 0.0000e+00
Epoch 4/30
24/24 [==============================] - 0s 693us/step - loss: -3413.0453 - accuracy: 0.0000e+00
Epoch 5/30
24/24 [==============================] - 0s 693us/step - loss: -6414.5028 - accuracy: 0.0000e+00
Epoch 6/30
24/24 [==============================] - 0s 742us/step - loss: -11650.4045 - accuracy: 0.0000e+00
Epoch 7/30
24/24 [==============================] - 0s 693us/step - loss: -20112.6117 - accuracy: 0.0000e+00
Epoch 8/30
24/24 [==============================] - 0s 780us/step - loss: -31282.6806 - accuracy: 0.0000e+00
Epoch 9/30
24/24 [==============================] - 0s 745us/step - loss: -47632.7406 - accuracy: 0.0000e+00
Epoch 10/30
24/24 [========

In [294]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8/8 - 0s - loss: -3.3695e+06 - accuracy: 0.0000e+00
Loss: -3369504.75, Accuracy: 0.0
